# Project Pythia Notebook Template

Let's start here! If you can directly link to an image relevant to your notebook, such as [canonical logos](https://github.com/numpy/numpy/blob/main/doc/source/_static/numpylogo.svg), do so here at the top of your notebook. You can do this with MyST Markdown syntax, outlined in [this MyST guide](https://mystmd.org/guide/figures), or you edit this cell to see a demonstration. **Be sure to include `alt` text for any embedded images to make your content more accessible.**

```{image} ../thumbnails/thumbnail.png
:alt: Project Pythia logo
:width: 200px
```

Next, title your notebook appropriately with a top-level Markdown header, `#` (see the very first cell above). Do not use this level header anywhere else in the notebook. Our book build process will use this title in the navbar, table of contents, etc. Keep it short, keep it descriptive. 

Follow this with a `---` cell to visually distinguish the transition to the prerequisites section.

---

## Overview
If you have an introductory paragraph, lead with it here! Keep it short and tied to your material, then be sure to continue into the required list of topics below,

1. This is a numbered list of the specific topics
1. These should map approximately to your main sections of content
1. Or each second-level, `##`, header in your notebook
1. Keep the size and scope of your notebook in check
1. And be sure to let the reader know up front the important concepts they'll be leaving with

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](https://foundations.projectpythia.org/core/cartopy/cartopy) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Time to learn**: estimate in minutes. For a rough idea, use 5 mins per subsection, 10 if longer; add these up for a total. Safer to round up and overestimate.
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports
Begin your body of content with another `---` divider before continuing into this section, then remove this body text and populate the following code cell with all necessary Python imports **up-front**:

In [ ]:
import xarray as xr
from decimal import Decimal
import hvplot.xarray
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import pelicanfs as pefs

## Initializing the datasets

This is where you begin your first section of material, loosely tied to your objectives stated up front. Tie together your notebook as a narrative, with interspersed Markdown text, images, and more as necessary,

In [ ]:
bucket_name = 'noaa-wcsd-zarr-pds'
ship_name = "Henry_B._Bigelow"
cruise_name = "HB1906"
sensor_name = "EK60"

zarr_store = f'{cruise_name}.zarr'
s3_zarr_store_path = f"{bucket_name}/level_2/{ship_name}/{cruise_name}/{sensor_name}/{zarr_store}"

cruise = xr.open_zarr(store=f"s3://{s3_zarr_store_path}", consolidated=True, storage_options={'anon': True})
start_time = "2019-10-12T17:30:00"
end_time = "2019-10-14T18:30:00"
timeslice = slice(start_time, end_time)
#
cruise = cruise.sel(time=timeslice, drop=False)
cruise = cruise.sel(frequency=38000, method='nearest')
print(cruise)
#
# print(select) # (4999, 599, 4)
# sv18_first = select.sel(frequency=18_000, depth=slice(0, 100)

#naive implementation of matching cruise coordinates and buoy coordinates
target_lon = 360 + cruise.longitude.values[0]
target_lat = cruise.latitude.values[0]
print(f"Target coordinates: Longitude: {target_lon}, Latitude: {target_lat}")

# ncar_ds = xr.open_dataset(
#     'osdf:///ncar/rda/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191006.nc', engine='netcdf4')
bouy_data_day_before = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191012.nc#mode=bytes', engine='netcdf4')
buoy_data_actual_day = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191013.nc#mode=bytes',
    engine='netcdf4')
buoy_data_day_after = xr.open_dataset(
    'https://data.rda.ucar.edu/d277007/avhrr_v2.1/2019/oisst-avhrr-v02r01.20191014.nc#mode=bytes',
    engine='netcdf4')
sst_day_before = bouy_data_day_before['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
sst_actual_day = buoy_data_actual_day['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]
sst_day_after = buoy_data_day_after['sst'].sel(lon=target_lon, lat=target_lat, method='nearest').values[0][0]

### Print daily sea surface temperature
Divide and conquer your objectives with Markdown subsections, which will populate the helpful navbar in Jupyter Lab and here on the Jupyter Book!

In [ ]:
print(sst_day_before)
print(sst_actual_day)
print(sst_day_after)

### Data Preprocessing
Keep up the good work! A note, *try to avoid using code comments as narrative*, and instead let them only exist as brief clarifications where necessary.

In [ ]:
baseline_timestamps = cruise.time.values
sst = np.full(shape=baseline_timestamps.shape, fill_value=np.nan)
df = pd.DataFrame(sst, columns=['sst'], index=baseline_timestamps)
df.loc[df.index[0],'sst'] = sst_day_before
df.loc[df.index[-1],'sst'] = sst_day_after
# print(int((len(df.index)/2)))
df.loc[df.index[int((len(df.index)/2))],'sst'] = sst_actual_day
df = df.interpolate(method='time')
print(df)

In [ ]:
def plot_synchronized_heatmaps(
        matrix1: np.ndarray,
        correlated_variable: np.ndarray,
        title1: str = "Sonar Data",
        title2: str = "Correlated Variable: Sea Surface Temperature",
        colorscale1: str = "Blues_r",
):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # --- Create Subplots ---
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        shared_yaxes=False,
        vertical_spacing=0.03
    )

    # --- Colorbar layout ---

    cb_len1, cb_y1 = 0.45, 0.775
    cb_len2, cb_y2 = 0.45, 0.225

    # --- Add Subplots ---
    fig.add_trace(
        go.Scatter(x=correlated_variable, y=correlated_variable**2),
        row=2, col=1
    )
    fig.add_trace(
        go.Heatmap(
            z=matrix1.astype(int),
            colorscale=colorscale1,
            zmin=matrix1.min(),
            zmax=matrix1.max(),
            colorbar_len=cb_len1,
            colorbar_y=cb_y1,
            name=title1,
            showscale=True,
            x=matrix1.time.values,
            y=matrix1.depth.values,
        ),
        row=1, col=1
    )
    # --- Layout Titles ---
    fig.update_layout(
        title_text="Signal Comparison",
        title_x=0.5,
        yaxis_title=title1,
        yaxis2_title=title2,
        yaxis_autorange='reversed',
        yaxis2_autorange='reversed'
    )

    # --- Explicit Axis Matching for Zoom Synchronization ---
    fig.update_layout(
        xaxis=dict(matches='x'),
        xaxis2=dict(matches='x'),
        yaxis=dict(matches='y'),
        yaxis2=dict(matches='y'),
    )

    fig.show()
    print("figure printed.")

plot_synchronized_heatmaps()

## Visualization
Here we can move on to our second objective, and we can demonstrate...

### A subsection to the second section

#### a quick demonstration

##### of further and further

###### header levels

as well as $m = a * t / h$ text! Similarly, you have access to other $\LaTeX$ equation [**functionality**](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html) via MathJax:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

Check out [**any number of helpful Markdown resources**](https://www.markdownguide.org/basic-syntax/) for further customizing your notebooks and the [**MyST Syntax Overview**](https://mystmd.org/guide/syntax-overview) for MyST-specific formatting information. Don't hesitate to ask questions if you have problems getting it to look *just right*.

## Last Section

You can add [admonitions using MyST syntax](https://mystmd.org/guide/admonitions):

:::{note}
Your relevant information here!
:::

Some other admonitions you can put in ([there are 10 total](https://mystmd.org/guide/admonitions#admonitions-list)):

:::{hint}
A helpful hint.
:::

:::{warning}
Be careful!
:::

:::{danger}
Scary stuff be here.
:::

We also suggest checking out Jupyter Book's [brief demonstration](https://jupyterbook.org/content/metadata.html#jupyter-cell-tags) on adding cell tags to your cells in Jupyter Notebook, Lab, or manually. Using these cell tags can allow you to [customize](https://jupyterbook.org/interactive/hiding.html) how your code content is displayed and even [demonstrate errors](https://jupyterbook.org/content/execute.html#dealing-with-code-that-raises-errors) without altogether crashing our loyal army of machines!

---

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!